In [24]:
import pyedflib
import os
import shutil
import glob
import datetime
import time
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

from numpy import mean, sqrt, square, arange
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [25]:
from scipy.signal import butter, sosfilt, sosfreqz


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y


# apply bandpass filter

fs = 25
lowcut = 0.55
highcut = 3.0

In [26]:
mdf = pd.read_csv(r'prof_data\resteaze\00000192-resteaze-sync.csv', sep=',')
mdf.head()

,Unnamed: 0,counter,unixTimes,accelerometerX,accelerometerY,accelerometerZ,capacitorsA,capacitorsB,capacitorsC,gyroscopeX,...,sleep_state,Leg Movement,Hypopnea,Arousal,RERA,Relative Desaturation,Obstructive Apnea,Central Apnea,Leg Movement (L),Leg Movement (R)
0,2568,2569,1645771253032,1353.0,-472.0,-3934.0,30734.0,NaN,NaN,-149.0,...,-1.0,0,0,0,0,0,0,0,0,0
1,2569,2574,1645771253039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0,0,0,0,0,0,0,0,0
2,2570,2570,1645771253072,1346.0,-486.0,-3924.0,30733.0,NaN,NaN,-142.0,...,-1.0,0,0,0,0,0,0,0,0,0
3,2571,2575,1645771253079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0,0,0,0,0,0,0,0,0
4,2572,2571,1645771253112,1342.0,-471.0,-3923.0,30733.0,NaN,NaN,-171.0,...,-1.0,0,0,0,0,0,0,0,0,0


In [30]:
mdf = mdf.apply(lambda x: pd.Series(x.dropna().values))

WK    322643
N2    140754
N3     55779
NS     33179
N1     19597
Name: sleep_stage, dtype: int64

In [31]:
all_df = mdf[mdf.sleep_state != -1].reset_index()

all_df = all_df[[
    'unixTimes', 'accelerometerX', 'accelerometerY', 'accelerometerZ',
    'gyroscopeX', 'gyroscopeY', 'gyroscopeZ', 'ledGreen',
    'sleep_stage', 'sleep_state'
]].dropna()

# Binary Classification
all_df["sleep_state"] = np.where(all_df["sleep_state"] == 0, 0, 1)

# RMS of accelerometer
all_df['accelerometer'] = all_df[[
    'accelerometerX', 'accelerometerY', 'accelerometerZ'
]].apply(lambda x: sqrt(square(x['accelerometerX']) + square(x['accelerometerY']) + square(x['accelerometerZ'])), axis=1)

# RMS of gyroscope
all_df['gyroscope'] = all_df[[
    'gyroscopeX', 'gyroscopeY', 'gyroscopeZ'
]].apply(lambda x: sqrt(square(x['gyroscopeX']) + square(x['gyroscopeY']) + square(x['gyroscopeZ'])), axis=1)

all_df = all_df.drop(columns=['accelerometerX', 'accelerometerY', 'accelerometerZ', 'gyroscopeX', 'gyroscopeY', 'gyroscopeZ'])

In [32]:
all_df['ledGreen'] = butter_bandpass_filter(all_df['ledGreen'],
                                            lowcut,
                                            highcut,
                                            fs,
                                            order=3)

In [33]:
all_df

,unixTimes,ledGreen,sleep_stage,sleep_state,accelerometer,gyroscope
0,1645771913034,119.082818,WK,0,4172.718418,297.546635
1,1645771913034,658.482111,WK,0,4176.995451,296.528245
2,1645771913074,1701.987486,WK,0,4182.306302,299.315887
3,1645771913074,2813.322086,WK,0,4182.463389,316.483807
4,1645771913114,3399.263287,WK,0,4171.734052,296.447297
...,...,...,...,...,...,...
250744,1645776902232,21.508890,WK,0,4198.440306,402.203928
250745,1645776902234,33.029222,WK,0,4199.829163,417.749925
250746,1645776902273,43.759520,WK,0,4196.294318,425.135273
250747,1645776902274,54.575285,WK,0,4205.252192,412.175933


In [34]:
all_df['sleep_stage'].value_counts()

WK    250749
Name: sleep_stage, dtype: int64

In [16]:
X = all_df.drop(['unixTimes', 'sleep_stage', 'sleep_state'], axis=1)
Y = all_df['sleep_state']
Y = LabelEncoder().fit_transform(Y)
X = StandardScaler().fit_transform(X)

In [17]:
# combine all the features
features = np.sum(X, axis=1)

In [18]:
features.shape

(250749,)

In [22]:
all_df['sleep_stage'].unique()

array(['WK'], dtype=object)